In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['SNS']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('sympathetic nervous system', 728),
 ('sympathetic nerve stimulation', 24),
 ('sacral nerve stimulation', 21),
 ('social networking sites', 9),
 ('sympathetic nerve system', 8),
 ('sini san', 7),
 ('sinisan', 6),
 ('secondary neurospheres', 4),
 ('sciatic nerve stimulation', 3),
 ('splanchnic nerve stimulation', 3),
 ('negative symptoms scale', 3),
 ('short nascent dna strands', 3)]

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-10-29 03:23:16] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'benzaldehyde lyase': 'MESH:C059416',
  'betaine aldehyde': 'CHEBI:CHEBI:15710',
  'bile salt activity lipase': 'HGNC:1848',
  'bioartificial liver': 'MESH:D019164',
  'blood alcohol levels': 'ungrounded',
  'breath alcohol levels': 'ungrounded',
  'british anti lewisite': 'CHEBI:CHEBI:64198',
  'brochoalveolar lavage': 'MESH:D018893',
  'bronchalveolar lavage': 'MESH:D018893',
  'bronchial alveolar lavage': 'MESH:D018893',
  'bronchial lavage': 'MESH:D018893',
  'bronchio alveolar lavage': 'MESH:D018893',
  'bronchiolar lavage': 'MESH:D018893',
  'broncho alveolar lavage': 'MESH:D018893',
  'bronchoalveolar': 'MESH:D018893',
  'bronchoalveolar fluid': 'MESH:D018893',
  'bronchoalveolar larvage': 'MESH:D018893',
  'bronchoalveolar lavage': 'MESH:D018893'},
 {'MESH:C059416': 'benzaldehyde lyase',
  'CHEBI:CHEBI:15710': 'betaine aldehyde',
  'HGNC:1848': 'CEL',
  'MESH:D019164': 'Liver, Artificial',
  'CHEBI:CHEBI:64198': 'dimercaprol',
  'MESH:D018893': 'Bronchoalveolar Lavage'},
 ['H

In [13]:
grounding_map, names, pos_labels = [{'sacral nerve stimulation': 'ungrounded',
  'secondary neurospheres': 'ungrounded',
  'serevent nationwide surveillance': 'ungrounded',
  'short nascent dna strands': 'ungrounded',
  'sini san': 'ungrounded',
  'sinisan': 'ungrounded',
  'social networking sites': 'ungrounded',
  'sympathetic': 'MESH:D013564',
  'sympathetic nerve stimulation': 'MESH:D013564',
  'sympathetic nerve system': 'MESH:D013564',
  'sympathetic nervous system': 'MESH:D013564'},
 {'MESH:D013564': 'Sympathetic Nervous System', 'HGNC:10582': 'SCN10A'},
 ['MESH:D013564', 'HGNC:10582']]

In [14]:
excluded_longforms = []

In [15]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [16]:
additional_entities = {'HGNC:10582': ['SCN10A', ['SNS']]}

In [17]:
unambiguous_agent_texts = {}

In [18]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [19]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [20]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [21]:
intersection1

[('HGNC:10582', 'HGNC:10582', 186)]

In [22]:
intersection2

[('MESH:D013564', 'HGNC:10582', 0), ('ungrounded', 'HGNC:10582', 0)]

In [23]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [24]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [25]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-29 03:30:32] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-29 03:30:39] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9836460516337551 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [26]:
classifier.stats

{'label_distribution': {'MESH:D013564': 615,
  'ungrounded': 32,
  'HGNC:10582': 8},
 'f1': {'mean': 0.983646, 'std': 0.005567},
 'precision': {'mean': 0.971981, 'std': 0.005953},
 'recall': {'mean': 0.9968, 'std': 0.003919},
 'MESH:D013564': {'f1': {'mean': 0.985587, 'std': 0.00315},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.971603, 'std': 0.006107}},
 'HGNC:10582': {'f1': {'mean': 0.866667, 'std': 0.163299},
  'pr': {'mean': 0.8, 'std': 0.244949},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.607576, 'std': 0.180831},
  'pr': {'mean': 0.461905, 'std': 0.151036},
  'rc': {'mean': 0.9, 'std': 0.2}}}

In [27]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [28]:
disamb.dump(model_name, results_path)

In [29]:
print(disamb.info())

Disambiguation model for SNS

Produces the disambiguations:
	SCN10A*	HGNC:10582
	Sympathetic Nervous System*	MESH:D013564

Class level metrics:
--------------------
Grounding                 	Count	F1     
Sympathetic Nervous System*	615	0.98559
                Ungrounded	 32	0.60758
                    SCN10A*	  8	0.86667

Weighted Metrics:
-----------------
	F1 score:	0.98365
	Precision:	0.97198
	Recall:		0.9968

* Positive labels
See Docstring for explanation



In [40]:
model_to_s3(disamb)

In [37]:
t = [x[0] for x in corpus if x[1].startswith('HGNC')]

In [39]:
print('\n\n'.join(t))

Neuropathic pain may be produced, at least in part, by the increased activity of primary afferent neurons. Studies have suggested that an accumulation of voltage-gated sodium channels at the site of peripheral nerve injury is a primary precursory event for subsequent afferent hyperexcitability. In this study, a human sodium channel (hPN3, SCN10A) has been cloned from the lumbar 4/5 dorsal root ganglia (DRG). Expression of hPN3 in Xenopus oocytes showed that this clone is a functional voltage-gated sodium channel. The amino acid sequence of hPN3 is most closely related to the rat PN3/SNS sodium channels which are expressed primarily in the small neurons of rat DRGs. The homologous relationship between rPN3 and hPN3 is defined by (i) a high level of sequence identity (ii) sodium currents that are highly resistant to tetrodotoxin (TTX) (iii) similar tissue distribution profiles and (iv) orthologous chromosomal map positions. Since rPN3/SNS has been implicated in nociceptive transmission, 